<h1 align="center">Hyperparamter Tunning</h1>

## Integrantes

- Josué Say
- Andre Jo

## Repositorio

- [Enlace a GitHub](https://github.com/JosueSay/labs_dl/tree/main/lab2)

In [ ]:
# %pip install -r requirements.txt

## Carga del Dataset

### Librerías y constantes

In [37]:
import os
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torch.utils.data import random_split
BASE_DIR = "./data/"
BATCH_SIZE = 64
FACTOR_VALIDATION_SET = 0.9
os.makedirs(BASE_DIR, exist_ok=True)

### Preprocesamiento

In [38]:
def getParamsTransform(bs):
    
    # El dataset de pytorch obtiene la data en formato PIL (Python Image Library) por lo que debemos pasarlo a 
    # tensores y que la red neuronal sea más fácil en procesar y luego transformamos sus datos de imágenes en rango 
    # `[0-255]` a `[0-1]`, esto sería una transformación base porque debemos al transformar datos debemos de actualizar 
    # la distribución y eso lo hacemos sabiendo la `mean` y `std`.
    base_transform = v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True)
    ])
     
    # Descargamos la data para train y test en caso que no este en `BASE_DIR` lo descargará sino solo lo leera de la carpeta `data`. Se aplica la transformación base.
    train_data = datasets.MNIST(root=BASE_DIR, train=True, download=True, transform=base_transform)
    
    # Cargamos los datos para aplicar una transformación para la carga de datos aplicando el batch_size (bs) dicho y si la data se queire revolver.
    train_loader = DataLoader(dataset=train_data, batch_size=bs, shuffle=True)
    
    mean = 0.0
    std = 0.0
    num_batches = 0

    # Se obtiene los valores exactos para usarlos en la transformación final
    for images, _ in train_loader:
        batch_mean = images.mean()
        batch_std = images.std()
        
        mean += batch_mean
        std += batch_std
        num_batches += 1

    mean /= num_batches
    std /= num_batches

    return mean, std

### Proceso de carga final

In [39]:
mean, std = getParamsTransform(bs=BATCH_SIZE)

final_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[mean], std=[std])
])

train_data = datasets.MNIST(root=BASE_DIR, train=True, download=False, transform=final_transform)
test_data = datasets.MNIST(root=BASE_DIR, train=False, download=False, transform=final_transform)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

print(f"Size dataset train {len(train_data)}")
print(f"Size dataset test {len(test_data)}")

Size dataset train 60000
Size dataset test 10000


Dado la `mean` y `std` ya es posible hacer la preparación de la data y se puede hacer el dataset para `validation`, por ejemplo 90% para entrenamiento y 10% para validación del dataset de train el cual es el que contiene más datos. Pero, puede editarse la proporción para el los datos de validation editanto `FACTOR_VALIDATION_SET`.

In [40]:
train_size = int(FACTOR_VALIDATION_SET * len(train_data))
val_size = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Al separar la data ya podemos utilizar los sets `train_loader`, `val_loader` y `test_loader`.

## Construcción del Modelo MLP

## Experimentación con Distintas Configuraciones

## Tuning de Hiperparámetros

## Evaluación del Modelo

## Referencias

- [Datasets PyTorch](https://docs.pytorch.org/vision/stable/datasets.html#mnist)
- [MNIST PyTorch](https://docs.pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html?highlight=mnist#torchvision.datasets.MNIST)
- [Transforming PyTorch](https://docs.pytorch.org/vision/stable/transforms.html#)
- [DataLoader PyTorch](https://docs.pytorch.org/docs/stable/data.html#)
- [Compose PyTorch](https://docs.pytorch.org/vision/stable/generated/torchvision.transforms.v2.Compose.html#torchvision.transforms.v2.Compose)